# Episteme Spacecraft — Minimal 1‑D Toy Ecosystem (Colab‑ready)

This single‑file notebook turns the Episteme equations into a simple, runnable loop over three roles of DNA:

1) **DNA as Data (Calibration)** — infer a feature map from genome to model parameters by matching simulated phenotypes.
2) **DNA as Algorithm (Evolution/Internalization)** — evolve genomes in varying habitats to internalize environment invariants.
3) **DNA as Operating System (Scheduling)** — compute a simple resource‑allocation utility guided by genome‑encoded controllers.

**Model sketch (1‑D):**
- Genome `S ∈ [0,1]^3` → features `θ = φ_η(S)`.
- Habitat field `E(x)` on a 1‑D line.
- Morphogen concentration `c(x,t)` evolves by a diffusion‑reaction PDE:
  $$\partial_t c = D(\theta)\,\partial_{xx} c + F_\theta(c,E)$$
- Chromatin state `s(t)` weakly feeds back on parameters: `\theta ← \theta ⊙ (1 + α_s s)`.
- Phenotype summary `Y(θ,E)` = simple features of the steady profile `c(x,T)`.
- Evolution uses a replicator–mutator update with fitness:
  $$\mathcal{F}(S,E)=R(Y(S,E)) + \kappa\,\mathrm{MI}(\Phi_η(S), G(E)) - \rho\,\text{cost}.$$
- OS utility (very simple) depends on controller rates derived from `θ`.

All code uses only NumPy, SciPy (optional fallback to NumPy), and Matplotlib. No seaborn. Each chart is a separate figure.


In [ ]:
#@title Imports & Utilities
import numpy as np
try:
    from scipy.stats import multivariate_normal
except Exception:
    multivariate_normal = None
import matplotlib.pyplot as plt
from dataclasses import dataclass
np.set_printoptions(precision=4, suppress=True)

def seed_all(seed=42):
    np.random.seed(seed)
seed_all(123)

def softplus(x):
    return np.log1p(np.exp(-np.abs(x))) + np.maximum(x,0)

def sigmoid(x):
    return 1/(1+np.exp(-x))

def softmax(z):
    z = z - np.max(z)
    e = np.exp(z)
    return e/np.sum(e)

def finite_diff_laplacian_1d(c, dx):
    # Neumann BCs
    lap = np.zeros_like(c)
    lap[1:-1] = (c[2:] - 2*c[1:-1] + c[:-2])/(dx*dx)
    lap[0] = (c[1] - c[0])/(dx*dx)
    lap[-1] = (c[-2] - c[-1])/(dx*dx)
    return lap


In [ ]:
#@title Model Components: Genome → Parameters → PDE → Phenotype
from dataclasses import dataclass
@dataclass
class Theta:
    D: float   # diffusion coeff
    a: float   # reaction growth coeff
    b: float   # reaction saturation coeff
    ctrl: np.ndarray  # 3 controller weights for OS scheduling

def phi_eta(S, eta):
    """
    Feature map θ = φ_η(S). Linear + sigmoid to keep parameters in range.
    S: (3,) genome in [0,1]
    eta: dict of mapping matrices
    """
    W = eta['W']  # shape (5,3): maps S→raw params [D,a,b,ctrl1,ctrl2], ctrl3 from bias
    b0 = eta['b']  # shape (5,)
    raw = W @ S + b0
    D = 1e-2 + 5e-2*sigmoid(raw[0])
    a = 0.05 + 0.6*sigmoid(raw[1])
    b = 0.4 + 0.6*sigmoid(raw[2])
    ctrl12 = sigmoid(raw[3:5])
    ctrl3 = sigmoid(eta.get('b_ctrl3', np.array([0.0])))[0]
    ctrl = np.array([ctrl12[0], ctrl12[1], ctrl3])
    return Theta(D=D, a=a, b=b, ctrl=ctrl)

def chromatin_update(s, c_mean, k_decay=0.05, k_coupling=0.1):
    ds = -k_decay*s + k_coupling*c_mean
    return s + ds

def apply_chromatin_feedback(theta: Theta, s, alpha_s=0.1):
    scale = 1 + alpha_s*s
    return Theta(D=theta.D*max(scale, 0.1), a=theta.a*max(scale, 0.1), b=theta.b, ctrl=theta.ctrl)

def habitat_field(Nx, kind='gradient'):
    x = np.linspace(0,1,Nx)
    if kind=='gradient':
        return x
    if kind=='bump':
        return np.exp(-((x-0.6)**2)/(2*0.03**2))
    if kind=='twopeaks':
        return np.exp(-((x-0.3)**2)/(2*0.04**2)) + 0.7*np.exp(-((x-0.75)**2)/(2*0.02**2))
    return x

def reaction_term(c, E, theta: Theta):
    # Simple logistic-type with habitat drive
    return theta.a*(c + 0.2*E) - theta.b*(c**2)

def simulate_pde(theta: Theta, E, dt=1e-3, T=2.0, dx=0.01, c0=None):
    Nx = len(E)
    steps = int(T/dt)
    if c0 is None:
        c = 0.1*np.ones(Nx)
    else:
        c = c0.copy()
    s = 0.0  # chromatin scalar
    for _ in range(steps):
        lap = finite_diff_laplacian_1d(c, dx)
        rc = reaction_term(c, E, theta)
        c = c + dt*(theta.D*lap + rc)
        c = np.maximum(c, 0.0)
        # Update chromatin and feedback occasionally
        s = chromatin_update(s, float(np.mean(c)))
        theta = apply_chromatin_feedback(theta, s, alpha_s=1e-3)
    return c

def phenotype_summary(c, E):
    # Phenotype Y: [mean concentration, center-of-mass, edge contrast]
    x = np.linspace(0,1,len(c))
    total = np.sum(c) + 1e-8
    com = float(np.sum(x*c)/total)
    mean_c = float(np.mean(c))
    edge = float(np.mean(np.abs(np.diff(c))))
    return np.array([mean_c, com, edge])

def distance_Y(Y, Yhat):
    w = np.array([1.0, 10.0, 1.0])  # emphasize center-of-mass alignment
    return float(np.sqrt(np.sum(w*(Y - Yhat)**2)))


In [ ]:
#@title Experiment 1: DNA as Data — Calibrate φ_η from synthetic pairs (S,E,Y)
def synth_ground_truth_theta(S):
    # Hidden true map for data generation (unknown to calibration)
    D = 0.02 + 0.03*(0.3*S[0] + 0.7*S[1])
    a = 0.15 + 0.5*S[2]
    b = 0.7 - 0.3*S[0]
    ctrl = np.array([0.5*S[0], 0.5*S[1], 0.5*S[2]])
    return Theta(D=D, a=a, b=b, ctrl=ctrl)

def make_dataset(n=20, Nx=101, dx=0.01):
    Ss, Es, Ys = [], [], []
    kinds = ['gradient','bump','twopeaks']
    for i in range(n):
        S = np.random.rand(3)
        E = habitat_field(Nx, kind=np.random.choice(kinds))
        th = synth_ground_truth_theta(S)
        c = simulate_pde(th, E, dt=5e-4, T=2.0, dx=dx)
        Y = phenotype_summary(c, E)
        Ss.append(S); Es.append(E); Ys.append(Y)
    return np.array(Ss), np.array(Es, dtype=object), np.array(Ys)

def init_eta():
    return {
        'W': 0.1*np.random.randn(5,3),
        'b': 0.1*np.random.randn(5),
        'b_ctrl3': np.array([0.0])
    }

def calibrate_eta(Ss, Es, Ys, eta, Nx=101, dx=0.01, iters=200):
    # Simple gradient-free: ES jitter on eta to minimize data misfit
    def total_loss(eta):
        loss = 0.0
        for S,E,Y in zip(Ss, Es, Ys):
            th = phi_eta(S, eta)
            c = simulate_pde(th, E, dt=5e-4, T=2.0, dx=dx)
            Yhat = phenotype_summary(c, E)
            loss += distance_Y(Y, Yhat)
        return loss/len(Ss)

    best = {k: v.copy() if isinstance(v, np.ndarray) else v for k,v in eta.items()}
    best_loss = total_loss(best)
    history = [best_loss]
    for t in range(iters):
        trial = {k: v + 0.05*np.random.randn(*v.shape) if isinstance(v, np.ndarray) else v for k,v in best.items()}
        L = total_loss(trial)
        if L < best_loss:
            best, best_loss = trial, L
        history.append(best_loss)
    return best, np.array(history)

# Generate data and calibrate
Ss, Es, Ys = make_dataset(n=15)
eta0 = init_eta()
eta_hat, hist = calibrate_eta(Ss, Es, Ys, eta0, iters=150)
print('Calibration loss (final):', hist[-1])

# Plot calibration loss
plt.figure()
plt.plot(hist)
plt.xlabel('Iteration')
plt.ylabel('Average data misfit')
plt.title('Experiment 1: Calibration loss')
plt.show()


In [ ]:
#@title Experiment 2: DNA as Algorithm — Evolution & Internalization
def phi_descriptor(S, eta):
    # Low‑dim descriptor of algorithmic choices from θ (e.g., a/D ratio and controller mix)
    th = phi_eta(S, eta)
    return np.array([th.a/(th.D+1e-6), th.ctrl[0], th.ctrl[1]])

def habitat_invariant(E):
    # Simple invariant: slope and peakiness
    slope = float(np.mean(np.diff(E)))
    peak = float(np.max(E) - np.min(E))
    return np.array([slope, peak, 1.0])

def gaussian_mi(X, Y):
    # Approximate MI for jointly Gaussian X,Y: I = 0.5 log |Σ_X||Σ_Y| / |Σ|
    X = X - X.mean(0, keepdims=True)
    Y = Y - Y.mean(0, keepdims=True)
    Z = np.concatenate([X,Y], axis=1)
    covX = np.cov(X, rowvar=False) + 1e-6*np.eye(X.shape[1])
    covY = np.cov(Y, rowvar=False) + 1e-6*np.eye(Y.shape[1])
    covZ = np.cov(Z, rowvar=False) + 1e-6*np.eye(Z.shape[1])
    signX, logdetX = np.linalg.slogdet(covX)
    signY, logdetY = np.linalg.slogdet(covY)
    signZ, logdetZ = np.linalg.slogdet(covZ)
    return 0.5*(logdetX + logdetY - logdetZ)

def fitness(S, E, eta, rho=0.01):
    # Reward from phenotype alignment to habitat (center of mass near E's peak)
    th = phi_eta(S, eta)
    c = simulate_pde(th, E, dt=5e-4, T=1.5, dx=0.01)
    Y = phenotype_summary(c, E)
    x = np.linspace(0,1,len(E))
    peak_x = x[np.argmax(E)]
    R_align = -abs(Y[1]-peak_x)
    cost = 0.1*th.D + 0.1*th.a + 0.05*np.sum(th.ctrl)
    return R_align - rho*cost, Y

def replicator_mutator(pop, fit, mu=0.05):
    w = np.exp(fit - np.max(fit))
    w = w/np.sum(w)
    idx = np.random.choice(len(pop), size=len(pop), p=w)
    children = pop[idx].copy()
    children += mu*np.random.randn(*children.shape)
    children = np.clip(children, 0.0, 1.0)
    return children

def run_evolution(generations=20, pop_size=40, Nx=101):
    pop = np.random.rand(pop_size,3)
    kinds = ['gradient','bump','twopeaks']
    mi_hist, fit_hist = [], []
    for g in range(generations):
        E = habitat_field(Nx, kind=np.random.choice(kinds))
        desc = np.array([phi_descriptor(S, eta_hat) for S in pop])
        inv = np.array([habitat_invariant(E) for _ in range(pop_size)])
        base_fit = []
        for S in pop:
            f,_ = fitness(S, E, eta_hat)
            base_fit.append(f)
        base_fit = np.array(base_fit)
        I = gaussian_mi(desc, inv)
        mi_hist.append(I)
        fit_hist.append(np.mean(base_fit))
        fit_plus = base_fit + 0.1*I
        pop = replicator_mutator(pop, fit_plus, mu=0.05)
    return np.array(mi_hist), np.array(fit_hist)

mi_hist, fit_hist = run_evolution()

# Plot MI over generations
plt.figure()
plt.plot(mi_hist)
plt.xlabel('Generation')
plt.ylabel('Approx. MI(Φ(S), G(E))')
plt.title('Experiment 2: Internalization metric')
plt.show()

# Plot average fitness over generations
plt.figure()
plt.plot(fit_hist)
plt.xlabel('Generation')
plt.ylabel('Mean fitness (no MI term)')
plt.title('Experiment 2: Base fitness trend')
plt.show()


In [ ]:
#@title Experiment 3: DNA as OS — Simple Scheduling Utility
def os_utility(theta: Theta, E):
    # Processes: growth, repair, division; allocate resources via softmax(ctrl)
    alloc = softmax(theta.ctrl)
    env = float(np.mean(E))
    R_growth = alloc[0]*(theta.a)*env
    R_repair = alloc[1]*(theta.b)*env
    R_risk = (1-alloc[2])*(0.1 + 0.5*(1-env))
    return R_growth + R_repair - 0.5*R_risk

def demo_os():
    Nx=101
    E = habitat_field(Nx, 'twopeaks')
    S = np.array([0.2,0.7,0.4])
    th = phi_eta(S, eta_hat)
    util = os_utility(th, E)
    c = simulate_pde(th, E, dt=5e-4, T=2.0, dx=0.01)
    x = np.linspace(0,1,Nx)
    plt.figure()
    plt.plot(x, E)
    plt.xlabel('x')
    plt.ylabel('E(x)')
    plt.title('Habitat field used in OS demo')
    plt.show()
    plt.figure()
    plt.plot(x, c)
    plt.xlabel('x')
    plt.ylabel('c(x,T)')
    plt.title('Steady morphogen profile — OS demo (utility={:.3f})'.format(util))
    plt.show()
    return util

u = demo_os()
print('OS utility (demo):', u)


In [ ]:
#@title Unified End‑to‑End Science Loop (toy)
def make_dataset(n=20, Nx=101, dx=0.01):
    Ss, Es, Ys = [], [], []
    kinds = ['gradient','bump','twopeaks']
    for i in range(n):
        S = np.random.rand(3)
        E = habitat_field(Nx, kind=np.random.choice(kinds))
        th = synth_ground_truth_theta(S)
        c = simulate_pde(th, E, dt=5e-4, T=2.0, dx=dx)
        Y = phenotype_summary(c, E)
        Ss.append(S); Es.append(E); Ys.append(Y)
    return np.array(Ss), np.array(Es, dtype=object), np.array(Ys)

def init_eta():
    return {
        'W': 0.1*np.random.randn(5,3),
        'b': 0.1*np.random.randn(5),
        'b_ctrl3': np.array([0.0])
    }

def calibrate_eta(Ss, Es, Ys, eta, Nx=101, dx=0.01, iters=100):
    def total_loss(eta):
        loss = 0.0
        for S,E,Y in zip(Ss, Es, Ys):
            th = phi_eta(S, eta)
            c = simulate_pde(th, E, dt=5e-4, T=2.0, dx=dx)
            Yhat = phenotype_summary(c, E)
            loss += distance_Y(Y, Yhat)
        return loss/len(Ss)
    best = {k: v.copy() if isinstance(v, np.ndarray) else v for k,v in eta.items()}
    best_loss = total_loss(best)
    history = [best_loss]
    for t in range(iters):
        trial = {k: v + 0.05*np.random.randn(*v.shape) if isinstance(v, np.ndarray) else v for k,v in best.items()}
        L = total_loss(trial)
        if L < best_loss:
            best, best_loss = trial, L
        history.append(best_loss)
    return best, np.array(history)

def science_loop(rounds=2, Nx=101):
    kinds = ['gradient','bump','twopeaks']
    metrics = []
    eta = {k:v.copy() if isinstance(v, np.ndarray) else v for k,v in eta_hat.items()}
    for r in range(rounds):
        Ss, Es, Ys = make_dataset(n=8, Nx=Nx)
        eta, hist = calibrate_eta(Ss, Es, Ys, eta, iters=60)
        mi_hist, fit_hist = run_evolution(generations=10, pop_size=30, Nx=Nx)
        E = habitat_field(Nx, kind=np.random.choice(kinds))
        S = np.random.rand(3)
        th = phi_eta(S, eta)
        util = os_utility(th, E)
        metrics.append({
            'cal_loss': float(hist[-1]),
            'mi_last': float(mi_hist[-1]),
            'fit_last': float(fit_hist[-1]),
            'os_util': float(util)
        })
        plt.figure()
        plt.plot(hist)
        plt.xlabel('Iteration')
        plt.ylabel('Calibration loss')
        plt.title(f'Round {r+1}: Calibration')
        plt.show()
        plt.figure()
        plt.plot(mi_hist)
        plt.xlabel('Generation')
        plt.ylabel('Approx. MI')
        plt.title(f'Round {r+1}: Internalization')
        plt.show()
        plt.figure()
        plt.plot(fit_hist)
        plt.xlabel('Generation')
        plt.ylabel('Mean base fitness')
        plt.title(f'Round {r+1}: Fitness trend')
        plt.show()
    return metrics

# Reuse calibration from Exp.1 as starting point
metrics = science_loop(rounds=2)
print('Metrics summary:', metrics)


### Notes
- **Stability:** The explicit Euler scheme is stable for the small `dt` used; adjust `dt`/`dx` if you change parameters.
- **Speed:** All steps are lightweight for Colab CPU. Increase grid or iterations for richer dynamics.
- **Extending:** Swap the reaction term for more complex kinetics, add measurement noise, or replace the MI approximation with a neural estimator.
